In [1]:
# designed to be run after 03-clinical_variables_final. this notebook does some data cleaning/processing. run before -___ notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#cohort import
from pathlib import Path
os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/04042019_final_pt_df2_v.csv', index_col=0) #final cohort for PA 

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 112 ms


# extracting clinical data for our patients
## IMPORTANT, USE THIS TO TUNE TIMEWINDOW OF EXTRACTION AND FOLDER TO SAVE IN

In [3]:
lower_window=0
upper_window=2
time_var="t_0"
#folder="48_hr_window"
date='16052019'

# date='04042019'
# lower_window=0
# upper_window=3
# time_var="t_0"
# folder="72_hr_window"

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_weightfirstday_df.csv'

time: 2.98 ms


### begin pipeline:

# changing my code structure to be a dictionary of dataframes

In [4]:
#folder to save files to:
save_path= wd+'/data/processed'
#folder=None


def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    global folder ##if bug here than need to try except
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    if add_subfolder==True:
        save_file=Path(save_path+'/%s/%s_%s_prepped.csv' %(folder,date, df_name))
        
        
    pd.DataFrame(df).to_csv(Path(save_path+'/%s_%s_prepped.csv' %(date, df_name)))
    del(df)

time: 10.1 ms


In [5]:
time_var='t_0' #for origional project time_var = 't_0', pa_project: 'first_pos_else_neg_ssc'

time: 921 µs


In [6]:


os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/'%(folder))
allFiles = glob.glob(os.getcwd() + "/{}_*.csv".format(date))

time: 2.83 ms


In [7]:
allFiles

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_bg_all_nosummary_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_uti_all_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_sofa_df_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_pt_info_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_vitals_all_nosummary_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_ventsettings_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_gcs72_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/04042019_labs_all_nosummary_72.csv',
 '/Users/geickelb1/D

time: 2.71 ms


In [8]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 15.6 s


In [9]:
#adding a t_0 to each df that doesn't currently have it

for element in df_list:
    #print(element,':',list(dfs[element]))
    if ('t_0' in list(dfs[element]))==False and 'icustay_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['icustay_id','t_0']], how='left')
    elif ('t_0' in list(dfs[element]))==False and 'hadm_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['hadm_id','t_0']], how='left')
    else:
        print("false")


false
false
false
false
false
false
false
false
false
false
false
time: 377 ms


In [10]:
for element in df_list:
    print(element,':',list(dfs[element]))

bg_all_nosummary_72 : ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'label', 'valuenum', 'value', 'valueuom', 'unique_var', 't_0', 'delta']
uti_all_72 : ['subject_id', 'hadm_id', 'itemid', 'charttime', 'value', 'valuenum', 'valueuom', 'label', 'fluid', 'category', 'loinc_code', 'icd9_code', 't_0', 'delta']
sofa_df_72 : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'sofa', 'respiration', 'pao2fio2_vent_min', 'pao2fio2_novent_min', 'coagulation', 'platelet_min', 'liver', 'bilirubin_max', 'cardiovascular', 'rate_dopamine', 'rate_epinephrine', 'rate_norepinephrine', 'rate_dobutamine', 'meanbp_min', 'cns', 'mingcs', 'renal', 'creatinine_max', 'urineoutput', 't_0', 'ICU_admit', 'approx_charttime']
pt_info_df : ['icustay_id', 'subject_id', 't_0', 'label', 'value', 'delta', 'uom']
vitals_all_nosummary_72 : ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'valueuom', 'vitalid', 'valuenum', 't_0', 'delta']
ventsettings_df : ['icustay_id', 'charttime', 'mechvent', 'oxygentherapy', 'ex

In [11]:
def yn_convert(df, #df in format where each row corresponds to a test, and a patient can have many rows
               label_fill, # value that will be filled to na's
               pt= final_pt_df2,
               time_var='t_0', #
               value_fill=0,
               delta_fill=pd.to_timedelta('0 days'),
               uom_fill='y/n'):

    """
    description: collapses (binarizes) a dataframe where each row corresponds to a test, and a patient can have many rows -> 
    1 row per patient where value is binary variable yes or no a patient has any value within the timewindow (specified in data collection).
    said a different way, for patient this fxn collapses values down to does pt have a non NA value in the clinical time window y/n? 
    
    label_fill: the variable name in the label column of the specified dataframe that will considered for y/n value within timewindow. if any non NA value is present
        it will be considered positive.
    pt: the by patient spreadsheet be be used to supply patient information.
    time_var: the variable used to create the time window of interest.
    value_fill: the variable value that missing values will be filled if the value is not present (default =0) in the origional dataset
    delta_fill: the time delta value that will be filled in if a patient doesn't have any instances of the label_fill.  
    uom_fill: fills in the unit of measurement to this for missing values.
    
    returns a flat 1 row per icustay_id of 1 or 0 if any value was present for the patient.
    """
    
    yn_df = pd.merge(pt[['icustay_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1 #squashes all other values into a binary 1 = yes
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)


time: 28.9 ms


## vaso dose

In [13]:
# #vaso_dose_72.head()
# vaso_dose_72= vaso_dose_72.rename(
#     columns={'starttime':'charttime','label':'vaso_type'}
#         )#.drop(labels='endtime', axis=1) 
# #renaming starttime to charttime and dropping endtime

dfs['vaso_dose_72']= dfs['vaso_dose_72'].rename(
    columns={'starttime':'charttime','label':'vaso_type'})

time: 34.5 ms


In [14]:
len(dfs['vaso_dose_72'])

50146

time: 1.85 ms


In [15]:
dfs['vaso_dose_72'].head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
8,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
9,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000
10,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000
22,200952.0,2139-09-23 23:12:00,2139-09-24 21:54:00,0.149992,7.990853,mg,mcg/kg/min,norepinephrine,2139-09-23,0 days 23:12:00.000000000


time: 30.7 ms


In [16]:
#removing units/hour because that is a different use of vasopressin
dfs['vaso_dose_72'] = dfs['vaso_dose_72'].loc[dfs['vaso_dose_72'].loc[:,'rate_uom']!= 'units/hour',:]
dfs['vaso_dose_72'] = dfs['vaso_dose_72'].loc[dfs['vaso_dose_72'].loc[:,'rate_uom']!= 'Uhr',:]

time: 27.8 ms


In [17]:
len(dfs['vaso_dose_72'])

50146

time: 3.81 ms


In [18]:
dfs['vaso_dose_72']['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 13.3 ms


#### removing outliers/extreme values

In [19]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:
def vaso_outlier_removal(df):
    test_group=(dfs['vaso_dose_72'][['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
         .groupby(['vaso_type','rate_uom'])
             )

    dfs['vaso_dose_72']['std']=test_group.transform(lambda x : x.std())
    dfs['vaso_dose_72']['mean']=test_group.transform(lambda x : x.mean())

    normal_high_value= pd.DataFrame({
        'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
        'high_value': [40, 20, 0.5, 1, 0.1, 2] #highest values one might expect to see in a clinic, ie above this is likely erroneous
    }) #found from literature, see notes

    dfs['vaso_dose_72'] = pd.merge(dfs['vaso_dose_72'], normal_high_value, left_on='vaso_type', right_on='vaso_type')

    vaso_dose_72_rmout =(dfs['vaso_dose_72'][
        ~((dfs['vaso_dose_72']['vaso_rate'] > dfs['vaso_dose_72']['high_value']) & ((dfs['vaso_dose_72']['vaso_rate']-dfs['vaso_dose_72']['mean'])>= (3*dfs['vaso_dose_72']['std'])))
        ])
    #ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean
    return(vaso_dose_72_rmout)

time: 18.4 ms


In [20]:
dfs['vaso_dose_72']= vaso_outlier_removal(dfs['vaso_dose_72'])
len(dfs['vaso_dose_72']) #52976 ->49340

49855

time: 124 ms


In [21]:
dfs['vaso_dose_72']['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 9.68 ms


In [22]:
#standardizing names, dropping unneeded columns for analysis
dfs['vaso_dose_72']= dfs['vaso_dose_72'].drop(['vaso_amount', 'amount_uom','std','mean','high_value'], axis=1)
dfs['vaso_dose_72']= dfs['vaso_dose_72'].rename(index=str, columns={'vaso_rate': 'value', 'rate_uom':'uom','vaso_type':'label'})
dfs['vaso_dose_72']['label'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 51 ms


In [23]:
dfs['vaso_dose_72'].head()

,icustay_id,charttime,endtime,value,uom,label,t_0,delta
0,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
1,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
2,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000
3,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000
4,201005.0,2169-10-25 17:25:00,2169-10-25 18:00:00,0.020000,mcgkgmin,epinephrine,2169-10-25,0 days 17:25:00.000000000


time: 30.6 ms


In [24]:
epinephrine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='epinephrine']
norepinephrine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='norepinephrine']
phenylephrine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='phenylephrine']
vasopressin_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='vasopressin']
dopamine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='dopamine']
dobutamine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='dobutamine']


time: 48.7 ms


In [25]:
#(could be transfered to a different spdsheet for collapsing values)
#y/n convert, seperating out vaso_dose into 6 constitutient dataframes, and for each am collapsing values down to does pt have in time window y/n? 

epinephrine_df=yn_convert(epinephrine_df, label_fill='epinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
norepinephrine_df=yn_convert(norepinephrine_df, label_fill='norepinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
phenylephrine_df=yn_convert(phenylephrine_df, label_fill='phenylephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
vasopressin_df=yn_convert(vasopressin_df, label_fill='vasopressin', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dopamine_df=yn_convert(dopamine_df, label_fill='dopamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dobutamine_df=yn_convert(dobutamine_df, label_fill='dobutamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)

time: 750 ms


In [26]:
save_path

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed'

time: 2.77 ms


In [27]:
pd.DataFrame(epinephrine_df).to_csv(Path(
    save_path+ '/%s_epinephrine_prepped.csv' %(date)))
del(epinephrine_df)

pd.DataFrame(norepinephrine_df).to_csv(Path(
    save_path+'/%s_norepinephrine_prepped.csv' %(date)))
del(norepinephrine_df)

pd.DataFrame(phenylephrine_df).to_csv(Path(
    save_path+'/%s_phenylephrine_prepped.csv' %(date)))
del(phenylephrine_df)

pd.DataFrame(vasopressin_df).to_csv(Path(
    save_path+'/%s_vasopressin_prepped.csv' %(date)))
del(vasopressin_df)

pd.DataFrame(dopamine_df).to_csv(Path(
    save_path+'/%s_dopamine_prepped.csv' %(date)))
del(dopamine_df)

pd.DataFrame(dobutamine_df).to_csv(Path(
    save_path+'/%s_dobutamine_prepped.csv' %(date)))
del(dobutamine_df)

del( dfs['vaso_dose_72'])


time: 2.28 s


## vitals -

In [28]:
dfs['vitals_all_nosummary_72'] = dfs['vitals_all_nosummary_72'].rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
dfs['vitals_all_nosummary_72'] = dfs['vitals_all_nosummary_72'].loc[dfs['vitals_all_nosummary_72']['label'].notnull(),:]#.count() #removing null values

dfs['vitals_all_nosummary_72'].loc[dfs['vitals_all_nosummary_72'].loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='RespRate') & 
    (dfs['vitals_all_nosummary_72']['uom']=='bpm'),'uom']='insp/min'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempC') & 
    (dfs['vitals_all_nosummary_72']['uom']=='?C'),'uom']='Deg. C'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempF') & 
    (dfs['vitals_all_nosummary_72']['uom']=='Deg. F'),'uom']='Deg. C'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempF') & 
    (dfs['vitals_all_nosummary_72']['uom']=='?F'),'uom']='Deg. C'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempF'),'label']='temperature'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempC'),'label']='temperature'

time: 13.5 s


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

In [29]:
#erroneous value cutoff summary
## setting a conservative threshold for erroneous values to not skew my data.

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'icustay_id']==228393.0) &
                         (dfs['vitals_all_nosummary_72'].loc[:,'label']=='Glucose') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']>99999), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='Glucose') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']>99998), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='Glucose') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<15), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='DiasBP') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<15), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='HeartRate') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value'].between(1,29)), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='RespRate') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<4), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='SysBP') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<40), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='TempC') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<28), 'value'])=np.nan

time: 3.44 s


In [30]:
dfs['vitals_all_nosummary_72'] = dfs['vitals_all_nosummary_72'].loc[dfs['vitals_all_nosummary_72']['value'].notnull(),:]#.count() #6930 NULL values


time: 1.77 s


In [31]:
save_df(dfs['vitals_all_nosummary_72'], 'vitals')
del(dfs['vitals_all_nosummary_72'])


time: 38.5 s


# labs -

In [12]:
dfs['labs_all_nosummary_72']= dfs['labs_all_nosummary_72'].rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 378 ms


In [13]:
dfs['labs_all_nosummary_72'].groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL       10555
ANION GAP    mEq/L      60646
BANDS        %           3468
BICARBONATE  mEq/L      61537
BILIRUBIN    mg/dL      17577
BUN          mg/dL      61976
CHLORIDE     mEq/L      67645
CREATININE   mg/dL      62241
GLUCOSE      mg/dL      79790
HEMATOCRIT   %          77302
HEMOGLOBIN   g/dL       63891
LACTATE      mmol/L     33788
PLATELET     K/uL       57124
POTASSIUM    mEq/L      84229
PT           SECONDS    38948
PTT          sec        40960
SODIUM       mEq/L      70562
WBC          K/uL       55480
Name: uom, dtype: int64

time: 329 ms


In [14]:
#outliers

time: 496 µs


In [15]:
#summary value removal- was explored and coded adhoc, difficult to automate
(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==261887) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='CHLORIDE')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==3.4),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==236290) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='CHLORIDE')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==11.0),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==292769) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==28.1),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==298457) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==48.8),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==234174) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==48.7),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==290264) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==42.0),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==290264) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==22.8),'value'])=np.nan

time: 642 ms


In [16]:
dfs['labs_all_nosummary_72']= dfs['labs_all_nosummary_72'].loc[dfs['labs_all_nosummary_72'].loc[:,'value'].notnull(),:] #removing null values


time: 105 ms


In [17]:
dfs['labs_all_nosummary_72'].head()

,subject_id,hadm_id,icustay_id,charttime,label,value,uom,t_0,delta
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,mEq/L,2149-11-13,1 days 07:47:00.000000000
6,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,mEq/L,2149-11-13,0 days 03:20:00.000000000
7,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,mEq/L,2149-11-13,0 days 16:04:00.000000000
8,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,mEq/L,2149-11-13,0 days 19:30:00.000000000
9,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,mEq/L,2149-11-13,1 days 01:48:00.000000000


time: 32.2 ms


In [18]:
#removing unwanted values
unwanted_values= ['HEMATOCRIT','ANION GAP','PT','ALBUMIN']
dfs['labs_all_nosummary_72']= dfs['labs_all_nosummary_72'].loc[~dfs['labs_all_nosummary_72'].loc[:,'label'].isin(unwanted_values),:]

time: 110 ms


In [36]:
def extreme_value_vis(df, label_values=None,value='valuenum', Min=True, Max=True, std_num=3):  
    """
    takes in a dataframe and list of value_labels and will return a list of all values that 
    lie outside of +/- 3 sd range for these groups 
    """
    cond_1= df.loc[:,value].notna()
    
    if label_values == None:
        
        cond_2= df.loc[:,'label']
    else:
        cond_2= df.loc[:,'label'].isin(label_values)
        
    cond_3= df.loc[:,value]> df.loc[:,value].mean()+ df.loc[:,value].std()

    df_outlier_explore= df[cond_1& cond_2].copy() #need to use .copy() to avoid valueset error
    df_outlier_explore['mean']=df_outlier_explore.groupby(['label'])[value].transform(lambda x : x.mean())
    df_outlier_explore['std']=df_outlier_explore.groupby(['label'])[value].transform(lambda x : x.std())

    df_outlier_explore['mean']=df_outlier_explore.groupby(['label'])[value].transform(lambda x : x.mean())

    if Max == True:
        cond4= (df_outlier_explore.loc[:,'value'].astype(float)>
                    (df_outlier_explore.loc[:,'mean'].astype(float)+std_num*df_outlier_explore.loc[:,'std'].astype(float)))
    else: cond4= False
    
    if Min == True:
        cond5= (df_outlier_explore.loc[:,'value'].astype(float)<
                    (df_outlier_explore.loc[:,'mean'].astype(float)-std_num*df_outlier_explore.loc[:,'std'].astype(float)))
    else: cond5= False
    return(df_outlier_explore.loc[cond4^cond5,:]) #rows that meet either condition but not both. 

time: 32.6 ms


In [37]:
dfs['labs_all_nosummary_72']
wbc_explore=extreme_value_vis(df=dfs['labs_all_nosummary_72'], label_values=['WBC'], value='value')

time: 93.5 ms


In [47]:
wbc_explore.sort_values(by='value', ascending=False) #wow 540 rows...

large_wbc=list(
    set(
        list(wbc_explore[wbc_explore['value']>100]['icustay_id']
            )
    )
) #96 rows above 100.

time: 3.94 ms


In [50]:
final_pt_df2[final_pt_df2['icustay_id'].isin(large_wbc)]['final_bin'].value_counts()

C_neg/A_full       19
C_pos/A_full        3
C_neg/A_partial     2
Name: final_bin, dtype: int64

time: 5.49 ms


In [39]:
pd.DataFrame(dfs['labs_all_nosummary_72']).to_csv(Path(
    save_path+'/%s_labs_prepped.csv' %(date)))
del(dfs['labs_all_nosummary_72'])

time: 5.94 s


## vent category -

In [40]:
dfs['ventcategory_df']['icustay_id'].nunique()
#13978 patients with someform of vent data. 

14068

time: 25.7 ms


In [41]:
dfs['ventcategory_df']['uom']='mech/O2/none category'
dfs['ventcategory_df'].head()

,icustay_id,day,value,uom,t_0
0,200012.0,0,Oxygen,mech/O2/none category,2153-12-23
1,200014.0,0,Mech,mech/O2/none category,2105-02-17
2,200014.0,1,Oxygen,mech/O2/none category,2105-02-17
3,200019.0,0,Mech,mech/O2/none category,2178-07-09
4,200019.0,1,Oxygen,mech/O2/none category,2178-07-09


time: 33.5 ms


In [42]:
dfs['ventcategory_df']=dfs['ventcategory_df'].rename(index=str, columns={'day':'delta'})
dfs['ventcategory_df']['label']='vent_recieved'
dfs['ventcategory_df']['delta']=pd.to_timedelta(dfs['ventcategory_df']['delta'], unit='d')
#dfs['ventcategory_df']= dfs['ventcategory_df'].drop(columns=['day'], axis=1) #removing day column

time: 44.9 ms


In [43]:
dfs['ventcategory_df'].head()

,icustay_id,delta,value,uom,t_0,label
0,200012.0,0 days,Oxygen,mech/O2/none category,2153-12-23,vent_recieved
1,200014.0,0 days,Mech,mech/O2/none category,2105-02-17,vent_recieved
2,200014.0,1 days,Oxygen,mech/O2/none category,2105-02-17,vent_recieved
3,200019.0,0 days,Mech,mech/O2/none category,2178-07-09,vent_recieved
4,200019.0,1 days,Oxygen,mech/O2/none category,2178-07-09,vent_recieved


time: 33 ms


In [44]:
#collapsing into 1 column for 3 days

time: 695 µs


In [45]:
def vent_day_collapser(x):
    if 'Mech' in list(x.unique()):
        x= 'Mech'
    elif 'Oxygen' in list(x.unique()):
        x= 'Oxygen'
    else:
        x='None'
        
    return(x)


time: 15.1 ms


In [46]:
#collapsing all days into the worst day. (could be transfered to a different spdsheet)
ventcategory_1day_df= dfs['ventcategory_df'].copy()
ventcategory_1day_df['value']=ventcategory_1day_df.groupby('icustay_id',as_index=False)['value'].transform(vent_day_collapser)
ventcategory_1day_df= ventcategory_1day_df.drop_duplicates(['icustay_id','value']).sort_values('icustay_id') #
ventcategory_1day_df= ventcategory_1day_df.loc[ventcategory_1day_df.loc[:,'icustay_id'].isin(icustay_id),:] #had icustay ids not in final cohort, fail safe mesure
#ventcategory_1day_df['label']= 'vent_recieved'
ventcategory_1day_df.head()

,icustay_id,delta,value,uom,t_0,label
0,200012.0,0 days,Oxygen,mech/O2/none category,2153-12-23,vent_recieved
1,200014.0,0 days,Mech,mech/O2/none category,2105-02-17,vent_recieved
3,200019.0,0 days,Mech,mech/O2/none category,2178-07-09,vent_recieved
6,200030.0,0 days,Oxygen,mech/O2/none category,2150-11-14,vent_recieved
9,200033.0,0 days,Mech,mech/O2/none category,2198-08-11,vent_recieved


time: 16.5 s


In [47]:
pd.DataFrame(ventcategory_1day_df).to_csv(Path(
    save_path+'/%s_ventcategory_prepped.csv' %(date)))
del(ventcategory_1day_df,dfs['ventcategory_df'] )

time: 267 ms


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [48]:
dfs['weightfirstday_df']['uom']='kg'

time: 1.44 ms


In [49]:
dfs['weightfirstday_df'].head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom,t_0
0,200012,51.2,51.200001,51.200001,NaN,NaN,kg,2153-12-23
1,200014,62.0,62.000000,NaN,NaN,NaN,kg,2105-02-17
2,200019,65.0,65.000000,NaN,NaN,NaN,kg,2178-07-09
3,200030,113.6,113.599998,113.599998,NaN,NaN,kg,2150-11-14
4,200033,74.0,74.000000,NaN,NaN,NaN,kg,2198-08-11


time: 17.8 ms


In [50]:
dfs['weightfirstday_df']= dfs['weightfirstday_df'][dfs['weightfirstday_df']['weight'].notnull()]
dfs['weightfirstday_df']= dfs['weightfirstday_df'][['icustay_id','weight','uom']]

time: 11.1 ms


In [51]:
dfs['weightfirstday_df']['label']= 'weight'
dfs['weightfirstday_df']=dfs['weightfirstday_df'].rename(index=str, columns={'weight':'value'})

#adding the assumed first day delta column to standardize all columns
dfs['weightfirstday_df']['delta']=pd.to_timedelta(0,'days')

#adding t_0
dfs['weightfirstday_df']= pd.merge(dfs['weightfirstday_df'], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 69.7 ms


In [52]:
pd.DataFrame(dfs['weightfirstday_df']).to_csv(Path(
    save_path+'/%s_weight_prepped.csv' %(date)))
del(dfs['weightfirstday_df'])

time: 310 ms


In [53]:
dfs['heightfirstday_df']['uom']='cm'

time: 15.9 ms


In [54]:
dfs['heightfirstday_df']= dfs['heightfirstday_df'][dfs['heightfirstday_df']['height'].notnull()]
dfs['heightfirstday_df']= dfs['heightfirstday_df'][['icustay_id','height','uom']]

time: 25.3 ms


In [55]:
dfs['heightfirstday_df']['label']= 'height'
dfs['heightfirstday_df']=dfs['heightfirstday_df'].rename(index=str, columns={'height':'value'})

#adding the assumed first day delta column to standardize all columns
dfs['heightfirstday_df']['delta']=pd.to_timedelta(0,'days')
#adding t_0
dfs['heightfirstday_df']= pd.merge(dfs['heightfirstday_df'], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 79.8 ms


In [56]:
# pd.DataFrame(dfs['heightfirstday_df']).to_csv(Path(
#     save_path+'/%s_height_prepped.csv' %(date)))
# del(dfs['heightfirstday_df'])

#heightfirstday
save_df(dfs['heightfirstday_df'], 'height')
del(dfs['heightfirstday_df'])

time: 229 ms


# UTI

In [57]:
dfs['uti_all_72']['value'].unique()#seems good #all uti within clinical timewindow

array(['NEG', 'MOD', 'SM', 'TR', 'POS', 'LG', 'SMALL', ' '], dtype=object)

time: 15.8 ms


In [58]:
dfs['uti_all_72']= dfs['uti_all_72'].loc[(dfs['uti_all_72'].loc[:,'value']!='NEG')&
               dfs['uti_all_72'].loc[:,'value'].notnull(),:] #filter to only pos rows

dfs['uti_all_72']= dfs['uti_all_72'].loc[(dfs['uti_all_72'].loc[:,'value']!='COMPUTER NETWORK FAILURE. TEST NOT RESULTED.')&
               dfs['uti_all_72'].loc[:,'value'].notnull(),:] #filter to only pos rows
dfs['uti_all_72']= dfs['uti_all_72'].drop_duplicates(subset=['hadm_id','value','charttime'])

dfs['uti_all_72'].loc[dfs['uti_all_72'].loc[:,'value'].notnull(),'value']= 1
dfs['uti_all_72'].loc[dfs['uti_all_72'].loc[:,'value'].isna(),'value']= 0 

time: 125 ms


In [59]:
dfs['uti_all_72'].head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
412,109,139061.0,51486,2141-09-11 09:15:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,40301,2141-09-11,0 days 09:15:00.000000000
869,191,136614.0,51486,2196-04-12 19:15:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,51881,2196-04-12,0 days 19:15:00.000000000
1024,202,108295.0,51486,2145-10-22 05:39:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,0389,2145-10-22,0 days 05:39:00.000000000
1649,256,153771.0,51486,2166-07-22 08:06:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,00845,2166-07-22,0 days 08:06:00.000000000
1812,307,132807.0,51486,2162-11-28 12:46:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,5845,2162-11-28,0 days 12:46:00.000000000


time: 36.9 ms


In [60]:
def uti_categorizer(uti_df):
    "useful to get all rows of days with positive values for patients (if multiple pos in a day there will be only 1 row for that day). ie more longitudinal format "
    
    #gives the max pos or neg value per day for 
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})  
    df_timewindow_perday= pd.merge(df_timewindow_perday,final_pt_df2[['icustay_id','hadm_id', time_var]], left_on='hadm_id', right_on='hadm_id', how='left')
    df_timewindow_perday=df_timewindow_perday.sort_values(['hadm_id','value','delta'], ascending=[True,False,True])
    return(df_timewindow_perday)

def uti_collapser(uti_df, label):
    "collapsing longitudinal data into 1 value. will return pos or neg if patient has a positive uti in their stay. one row per icustay_id"
    
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})
    first_pos= df_timewindow_perday.drop_duplicates(['hadm_id'])
    collapsed= pd.merge(final_pt_df2[['hadm_id','icustay_id','subject_id', time_var]],first_pos, left_on='hadm_id', right_on='hadm_id', how='left')
    
    collapsed['value']= collapsed['value'].fillna(0)
    collapsed.loc[collapsed.loc[:,'value']==1,'value']= 'pos'
    collapsed.loc[collapsed.loc[:,'value']==0,'value']= 'Neg/Not_tested'
    
    collapsed['delta']= collapsed['delta'].fillna(pd.Timedelta(1, unit='d'))
    
    collapsed['label']= label
    
    collapsed['uom']='pos/neg category'
    return(collapsed)

time: 65.9 ms


In [61]:
uti_nit_pos= dfs['uti_all_72'][dfs['uti_all_72']['label']=="Nitrite"]
uti_leuk_pos= dfs['uti_all_72'][dfs['uti_all_72']['label']=="Leukocytes"]

leuk_collapsed= uti_collapser(uti_leuk_pos, 'leukocyte')
nit_collapsed= uti_collapser(uti_nit_pos, 'nitrite')


time: 105 ms


In [62]:
save_df(leuk_collapsed, 'leuk')
del(leuk_collapsed)

save_df(nit_collapsed, 'nit')
del(nit_collapsed)
del(dfs['uti_all_72'])

time: 434 ms


# bloodgas

In [11]:

dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72']['value'].notnull(),:]
dfs['bg_all_nosummary_72'] = dfs['bg_all_nosummary_72'].rename(index=str, columns={'valueuom':'uom'})
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[~(dfs['bg_all_nosummary_72'].loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
dfs['bg_all_nosummary_72'].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
82,9,150750,220597,2149-11-13 03:34:00,BASEEXCESS,NaN,-1,mEq/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
83,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
84,9,150750,220597,2149-11-13 03:34:00,LACTATE,1.80,1.8,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
85,9,150750,220597,2149-11-13 03:34:00,PCO2,34.00,34,mm Hg,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
86,9,150750,220597,2149-11-13 03:34:00,PH,7.42,7.42,units,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000


time: 1.05 s


#### erroneous value removal

NOTE: these are hard coded to positioning. if anything in the bg_all_nosummary is changed than these positions need to be updated. 

In [12]:
##calcium
#fixing the calcium errors w/o hard coding
(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==249571) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==249571) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==219600) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==219600) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 476 ms


In [13]:
##min chloride
#converting it to a null value without hard coding
(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==261887.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==261887.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==236290.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==236290.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==11.0),'value'])=np.nan

time: 303 ms


In [14]:
#peep changes summary: 
dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='PEEP')&
                        (dfs['bg_all_nosummary_72']['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 68.8 ms


In [15]:
#temp changes summary:

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==253821)&
                        (dfs['bg_all_nosummary_72']['valuenum']==18.9),
                        'value']= np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==253821)&
                        (dfs['bg_all_nosummary_72']['valuenum']==18.9),
                        'valuenum']= np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==251788)&
                        (dfs['bg_all_nosummary_72']['valuenum']==10.0),
                        'value']= np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==251788)&
                        (dfs['bg_all_nosummary_72']['valuenum']==10.0),
                        'valuenum']= np.nan

time: 329 ms


In [16]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='FIO2') &
                        (dfs['bg_all_nosummary_72'].loc[:,'value']=='0'),'value']=np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='FIO2') &
                        (dfs['bg_all_nosummary_72'].loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(0.0,1.0)),'valuenum']

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 425 ms


#### removing null values annotated abov

In [17]:
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72']['value'].notnull(),:]

time: 77.3 ms


### splitting specimen out for vent vs non-vent bg data

In [18]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72'].loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

dfs['bg_all_nosummary_72']= pd.merge(dfs['bg_all_nosummary_72'],specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary_72=dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72'].loc[:,'specimen']=='ART',:]
del(specimen_df)

time: 698 ms


In [19]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE']
bg_vent_labels=['PCO2','PaO2','PO2','FIO2','PEEP','O2FLOW']

time: 1.3 ms


In [20]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72'].loc[:,'label'].isin(bg_labels),:]
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label'].isin(bg_vent_labels),:]

time: 106 ms


In [21]:
dfs['bg_all_nosummary_72'].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta,specimen
1,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000,ART
2,9,150750,220597,2149-11-13 03:34:00,LACTATE,1.80,1.8,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000,ART
4,9,150750,220597,2149-11-13 03:34:00,PH,7.42,7.42,units,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000,ART
7,9,150750,220597,2149-11-13 03:34:00,TEMPERATURE,37.20,37.2,NaN,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000,ART
14,9,150750,220597,2149-11-13 08:07:00,PH,7.42,7.42,units,2205972149-11-13 08:07:00,2149-11-13,0 days 08:07:00.000000000,ART


time: 18.3 ms


In [22]:
bg_col=['subject_id','hadm_id','icustay_id','charttime','delta',time_var,'label','valuenum','uom']
bg_ART_nosummary_72= bg_ART_nosummary_72[bg_col]
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'][bg_col]
del(bg_col)

time: 25.8 ms


In [23]:
bg_ART_nosummary_72['label'].unique()

array(['PCO2', 'PaO2', 'FIO2', 'PEEP', 'O2FLOW'], dtype=object)

time: 14 ms


In [24]:
#converting O2 to y/n
o2_flow_df= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='O2FLOW',:]
o2_flow_df=o2_flow_df.rename(index=str, columns={'valuenum':'value'})
o2_flow_df= yn_convert(o2_flow_df, label_fill='o2_flow', time_var=time_var)
o2_flow_df['label']="o2_flow" #fixing label

#removing o2_flow from bg_ART
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='O2FLOW',:]

time: 178 ms


In [25]:
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='PEEP',:]

time: 26.6 ms


The PaO2/FiO2 ratio should be calculated using values that are recorded either at the same time stamp, or when the FIO2 occurs within 2 hours BEFORE the PaO2, not the other way around.
For those with no FiO2 recorded within that time frame, you can assume it’s 0.21 (=21%). The worse ratio will be the lowest PaO2 recorded with FiO2 of 1.0 (=100%). So if that is 12, then it will be 12/1 = 12.

In [26]:
#need to groupby icustay_id and charttime for both. pan

pao2= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='PaO2',:].copy() #n=41799 non-null
fio2= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='FIO2',:].copy() #n=12311 non-null
pao2['delta']=pd.to_timedelta(pao2['delta'])
fio2['delta']=pd.to_timedelta(fio2['delta'])
#fio2['offset']=fio2['delta'] + pd.to_timedelta(2, unit='h')
pao2['offset']=pao2['delta'] - pd.to_timedelta(2, unit='h') #pao2 should have the offset, not fio2
#pafi_merge=pd.merge(pao2, fio2[['icustay_id','delta','valuenum','uom']], on=['icustay_id','delta'], how='left')

#pafi_merge.loc[pafi_merge.loc[:,'valuenum_y'].notnull(),:] #12310 fio2 values merged over. this is great news, all but 1 have an associated PAO2.
#now need to find teh pao2 values without exact match and find the fio2 within 2 hr.
#if the row is within 2 hours of another row, merge.


time: 266 ms


In [27]:
#removing pao2 and fio2 from bg_ART_nosummary_72
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='PaO2',:]
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='FIO2',:]



time: 34.8 ms


In [28]:
import pandasql as ps

sqlcode = '''
SELECT A.*, B.valuenum as valuenum2, B.label as label2, B.charttime as charttime2
FROM pao2 as A --grabs from both merged and non-merged as is, or jsut from notmerged if chagned to pao2_notmerged
LEFT JOIN fio2 AS B on A.icustay_id = B.icustay_id
WHERE B.delta BETWEEN A.offset and A.delta;
'''

newdf = ps.sqldf(sqlcode,locals())

#(test_rows.delta >= fio2.delta and test_rows.delta<= fio2.offset)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandasql/sqldf.py:121: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  index=not any(name is None for name in df.index.names))  # load index into db if all levels are named
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandasql/sqldf.py:121: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  index=not any(name is None for name in df.index.names))  # load index into db if all levels are named
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandasql/sqldf.py:121: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  index=not any(name is None for name in df.index.names))  # load index into db if all levels are named


time: 890 ms


In [29]:
#newdf #15196
#1040 rows from notmerged. 
#grabes from both
newdf['delta']=pd.to_timedelta(newdf['delta'])
newdf['offset']=pd.to_timedelta(newdf['delta'])
ratio_df= pd.merge(pao2, newdf[['icustay_id','delta','valuenum2', 'charttime2']], on=['icustay_id','delta'], how='left')
del(newdf, pao2,fio2)


time: 27.8 ms


In [30]:
#i merged in all rows that matched the 2hr criteria
## For those with no FiO2 recorded within that time frame, you can assume it’s 0.21 (=21%). 
ratio_df.loc[ratio_df.loc[:,'valuenum2'].isnull(),'valuenum2']= 21
ratio_df['ratio']= ratio_df['valuenum']/(ratio_df['valuenum2']/100)
ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime2']= ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime']
ratio_df['deltadelta']=pd.to_datetime(ratio_df['charttime'])-pd.to_datetime(ratio_df['charttime2'])


time: 77.8 ms


In [31]:
ratio_df['ratio'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    41324.000000
mean       497.368856
std        377.415510
min         22.000000
10%        150.000000
25%        260.000000
50%        412.000000
75%        600.000000
99%       1995.238095
max       3690.476190
Name: ratio, dtype: float64

time: 12.8 ms


In [32]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    41324.000000
mean       131.441729
std         76.914515
min         15.000000
10%         69.000000
25%         84.000000
50%        109.000000
75%        149.000000
99%        436.000000
max        775.000000
Name: valuenum, dtype: float64

time: 11.3 ms


In [33]:
### all of the duplicate PaO2 with different FIo2. if a pao2 value has 2 fio2 values that match the 2 hour window, then the one closest time to the fio2 value is chosen. 
ratio_df=ratio_df.sort_values(['icustay_id','delta','deltadelta']).drop_duplicates(['icustay_id','delta','ratio'])
#43715 ->42455

time: 32.4 ms


In [34]:
ratio_df['ratio'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    40126.000000
mean       506.060795
std        379.019545
min         22.000000
10%        156.000000
25%        271.428571
50%        419.047619
75%        609.523810
99%       2004.761905
max       3690.476190
Name: ratio, dtype: float64

time: 11.1 ms


In [35]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    40126.000000
mean       131.801949
std         77.198487
min         15.000000
10%         69.000000
25%         84.000000
50%        109.000000
75%        150.000000
99%        437.750000
max        775.000000
Name: valuenum, dtype: float64

time: 12.3 ms


In [36]:
#finding worst value:

time: 538 µs


In [37]:
ratio_df[ratio_df['icustay_id']==200014]

,subject_id,hadm_id,icustay_id,charttime,delta,t_0,label,valuenum,uom,offset,valuenum2,charttime2,ratio,deltadelta
8263,9514,127229,200014,2105-02-17 01:08:00,0 days 01:08:00,2105-02-17,PaO2,179.0,mm Hg,-1 days +23:08:00,21.0,2105-02-17 01:08:00,852.380952,00:00:00
8264,9514,127229,200014,2105-02-17 04:35:00,0 days 04:35:00,2105-02-17,PaO2,183.0,mm Hg,0 days 02:35:00,21.0,2105-02-17 04:35:00,871.428571,00:00:00
8265,9514,127229,200014,2105-02-17 11:18:00,0 days 11:18:00,2105-02-17,PaO2,142.0,mm Hg,0 days 09:18:00,21.0,2105-02-17 11:18:00,676.190476,00:00:00
8266,9514,127229,200014,2105-02-17 15:28:00,0 days 15:28:00,2105-02-17,PaO2,155.0,mm Hg,0 days 13:28:00,40.0,2105-02-17 15:28:00,387.500000,00:00:00
8268,9514,127229,200014,2105-02-17 17:11:00,0 days 17:11:00,2105-02-17,PaO2,96.0,mm Hg,0 days 15:11:00,28.0,2105-02-17 17:11:00,342.857143,00:00:00
8267,9514,127229,200014,2105-02-17 17:11:00,0 days 17:11:00,2105-02-17,PaO2,96.0,mm Hg,0 days 15:11:00,40.0,2105-02-17 15:28:00,240.000000,01:43:00
8269,9514,127229,200014,2105-02-17 20:53:00,0 days 20:53:00,2105-02-17,PaO2,95.0,mm Hg,0 days 18:53:00,21.0,2105-02-17 20:53:00,452.380952,00:00:00
8270,9514,127229,200014,2105-02-18 04:01:00,1 days 04:01:00,2105-02-17,PaO2,101.0,mm Hg,1 days 02:01:00,21.0,2105-02-18 04:01:00,480.952381,00:00:00


time: 30.3 ms


In [38]:
# ratio_df.groupby(['icustay_id'], as_index=False)['ratio'].min() #is this what i want for my pao2/fio2 ratio?


f = {'ratio': 'min','delta':'first'}
ratio_min_df=ratio_df.sort_values(['icustay_id','ratio'],ascending=True).groupby('icustay_id', as_index=False).agg(f)

##it's likely that this variable will not explain any additional variation in y than the ventilation category variable we already have. 
##maybe we can stratify patients with a few categories:

##not on vent
##being on vent with normal pao2/fio2 ratio
## being on vent with low pao2/fio2 ratio

time: 35.6 ms


In [39]:
ratio_min_df.head()

,icustay_id,ratio,delta
0,200014,240.000000,0 days 17:11:00
1,200019,447.619048,1 days 05:28:00
2,200030,357.142857,2 days 22:34:00
3,200059,312.000000,0 days 07:41:00
4,200065,98.000000,0 days 08:27:00


time: 7.28 ms


In [40]:
final_pt_df2.head()

,subject_id,hadm_id,t_0_sc,icustay_id,ab_course,t_0,t_end_consec,ICU_admit,dod_hosp,dod_ssn,dod_delta,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin
0,3301,194119,2116-06-27 00:00:00,222957.0,full,2116-06-28,2116-06-28,2116-06-26 20:05:51,NaN,NaN,NaN,NaN,NaN,2116-06-27 00:00:00,negative,C_neg/A_full
1,3281,113868,2115-09-14 19:25:00,240363.0,partial,2115-09-14,2115-09-16,2115-09-13 23:16:50,2116-03-06 00:00:00,2116-03-06 00:00:00,0 days 00:00:00.000000000,NaN,NaN,2115-09-14 19:25:00,negative,C_neg/A_partial
2,3277,163302,2123-07-24 09:30:00,220265.0,full,2123-07-24,2123-08-06,2123-07-21 13:04:46,NaN,NaN,NaN,NaN,NaN,2123-07-24 09:30:00,negative,C_neg/A_full
3,3276,111860,2168-08-31 00:00:00,224362.0,partial,2168-08-30,2168-08-30,2168-08-26 21:21:59,NaN,NaN,NaN,NaN,NaN,2168-08-31 00:00:00,negative,C_neg/A_partial
6,3272,105931,2114-11-20 15:50:00,296728.0,partial,2114-11-21,2114-11-23,2114-11-20 19:51:15,NaN,NaN,NaN,NaN,NaN,2114-11-20 15:50:00,negative,C_neg/A_partial


time: 17.4 ms


In [41]:
len(ratio_min_df)

5352

time: 1.92 ms


In [42]:
ratio_min_df= pd.merge(final_pt_df2[['icustay_id','hadm_id','t_0',]],ratio_min_df, left_on='icustay_id',right_on='icustay_id', how='left')
ratio_min_df= ratio_min_df.rename(index=str, columns={'ratio':'value'})

ratio_min_df.loc[ratio_min_df.loc[:,'value'].isnull(),'value']=476

ratio_min_df['uom']= 'PaO2/FIO2 ratio'
ratio_min_df['label']= 'PaO2/FIO2'

time: 31.4 ms


In [43]:
ratio_min_df.loc[:,'delta']=ratio_min_df.loc[:,'delta'].fillna(pd.to_timedelta('0 days'))

time: 9.8 ms


In [44]:
ratio_min_df['value'].describe()#100 PaO2 on 21% FiO2 = 476 ratio

count    14478.000000
mean       413.712727
std        154.096198
min         22.000000
25%        357.142857
50%        476.000000
75%        476.000000
max       2490.476190
Name: value, dtype: float64

time: 7.25 ms


In [45]:
ratio_min_df #100 PaO2 on 21% FiO2 = 476 ratio

,icustay_id,hadm_id,t_0,value,delta,uom,label
0,222957.0,194119,2116-06-28,438.095238,0 days 02:39:00,PaO2/FIO2 ratio,PaO2/FIO2
1,240363.0,113868,2115-09-14,476.000000,0 days 00:00:00,PaO2/FIO2 ratio,PaO2/FIO2
2,220265.0,163302,2123-07-24,390.476190,0 days 11:07:00,PaO2/FIO2 ratio,PaO2/FIO2
3,224362.0,111860,2168-08-30,476.000000,0 days 00:00:00,PaO2/FIO2 ratio,PaO2/FIO2
4,296728.0,105931,2114-11-21,476.000000,0 days 00:00:00,PaO2/FIO2 ratio,PaO2/FIO2
5,278829.0,134370,2193-08-28,400.000000,0 days 10:25:00,PaO2/FIO2 ratio,PaO2/FIO2
6,271485.0,156533,2137-04-04,476.000000,0 days 00:00:00,PaO2/FIO2 ratio,PaO2/FIO2
7,206821.0,155346,2169-10-08,140.000000,1 days 13:06:00,PaO2/FIO2 ratio,PaO2/FIO2
8,257253.0,155197,2130-07-20,585.714286,0 days 00:42:00,PaO2/FIO2 ratio,PaO2/FIO2
9,275841.0,146611,2150-04-06,51.000000,1 days 08:06:00,PaO2/FIO2 ratio,PaO2/FIO2


time: 30.5 ms


In [49]:
save_df(ratio_min_df,'pafaRatio') # minimum pao2:fio2 ratio during a stay
del(ratio_min_df)

save_df(dfs['bg_all_nosummary_72'],'bg_all') #all bloodgas
del(dfs['bg_all_nosummary_72'])

save_df(bg_ART_nosummary_72,'bg_ART') #only ARTERIAL bloodgas
del(bg_ART_nosummary_72)

save_df(o2_flow_df,'o2_flow') #need to investigate this more
del(o2_flow_df)

time: 2.51 s


# RRT

In [99]:
#removing null values
dfs['rrt_merged_allpt_df'] = dfs['rrt_merged_allpt_df'].loc[dfs['rrt_merged_allpt_df']['rrt'].notnull(),:]
dfs['rrt_merged_allpt_df']['uom']='category'
dfs['rrt_merged_allpt_df']['delta']=pd.to_timedelta(
    pd.to_datetime(dfs['rrt_merged_allpt_df']['first_charttime'])-
    pd.to_datetime(dfs['rrt_merged_allpt_df'][time_var]),
    'days')
dfs['rrt_merged_allpt_df']['label']= 'rrt'
dfs['rrt_merged_allpt_df']=dfs['rrt_merged_allpt_df'].rename(index=str, columns={'rrt':'value'})

time: 39.2 ms


### converting to yes/no

In [100]:
dfs['rrt_merged_allpt_df'].head()

,icustay_id,t_0,first_charttime,value,uom,delta,label
0,222957.0,2116-06-28,NaN,0.0,category,NaT,rrt
1,240363.0,2115-09-14,NaN,0.0,category,NaT,rrt
2,220265.0,2123-07-24,NaN,0.0,category,NaT,rrt
3,224362.0,2168-08-30,NaN,0.0,category,NaT,rrt
4,296728.0,2114-11-21,NaN,0.0,category,NaT,rrt


time: 17.2 ms


In [101]:
dfs['rrt_merged_allpt_df']['icustay_id'].nunique() #

14478

time: 27.1 ms


In [102]:
rrt_yn= yn_convert(dfs['rrt_merged_allpt_df'],label_fill='rrt', time_var=time_var)

time: 41.8 ms


In [103]:
save_df(rrt_yn, 'rrt')
del(rrt_yn, dfs['rrt_merged_allpt_df'])

time: 321 ms


# GCS_72

In [104]:
dfs['gcs72_df']['uom']='GCS_score'

time: 1.84 ms


In [105]:
list(dfs['gcs72_df'])

['subject_id',
 'hadm_id',
 'icustay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'ICU_admit',
 't_0',
 'approx_charttime',
 'admit_plus_day',
 'delta',
 'uom']

time: 3.29 ms


In [106]:
dfs['gcs72_df']['label']= 'mingcs'
dfs['gcs72_df']['uom']='gcs_score'
dfs['gcs72_df']=dfs['gcs72_df'].rename(index=str, columns={'mingcs':'value'})

time: 20.1 ms


In [107]:
dfs['gcs72_df']=dfs['gcs72_df'][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 3.74 ms


In [108]:
save_df(dfs['gcs72_df'], 'gcs')
del(dfs['gcs72_df'])

time: 73 ms


# SOFA_72
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [109]:
dfs['sofa_df_72']['uom']='daily_sofa_score'

#adding day delta column
dfs['sofa_df_72']=dfs['sofa_df_72'].sort_values(['hadm_id','day',time_var]) #good
dfs['sofa_df_72']['day_rank']=dfs['sofa_df_72'].groupby('icustay_id')['day'].rank()
dfs['sofa_df_72']['delta']=pd.to_timedelta((dfs['sofa_df_72']['day_rank']-1), 'days')

time: 203 ms


In [110]:
dfs['sofa_df_72']['label']= 'daily_sofa'
dfs['sofa_df_72']=dfs['sofa_df_72'].rename(index=str, columns={'sofa':'value'})

time: 76.2 ms


In [111]:
dfs['sofa_df_72']= dfs['sofa_df_72'][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 24.5 ms


In [112]:
dfs['sofa_df_72'].head()

,subject_id,hadm_id,icustay_id,delta,label,value,t_0,uom
57823,54610,100003,209281,0 days,daily_sofa,4,2150-04-18,daily_sofa_score
32233,23018,100007,217937,0 days,daily_sofa,1,2145-03-31,daily_sofa_score
32234,23018,100007,217937,1 days,daily_sofa,4,2145-03-31,daily_sofa_score
32235,23018,100007,217937,2 days,daily_sofa,0,2145-03-31,daily_sofa_score
32236,23018,100007,217937,3 days,daily_sofa,3,2145-03-31,daily_sofa_score


time: 37.8 ms


In [113]:
save_df(dfs['sofa_df_72'], 'sofa')
del(dfs['sofa_df_72'])

time: 889 ms


# patient Demographic variables

#completed in 03.01 clinical_variable

In [114]:
dfs['pt_info_df']

,icustay_id,subject_id,t_0,label,value,delta,uom
0,220597.0,9,2149-11-13,first_admit_age,41.79,0 days 00:00:00.000000000,years
1,273430.0,19,2108-08-06,first_admit_age,300.0,0 days 00:00:00.000000000,years
2,203487.0,25,2160-11-02,first_admit_age,58.95,0 days 00:00:00.000000000,years
3,244882.0,26,2126-05-10,first_admit_age,72.0,0 days 00:00:00.000000000,years
4,296681.0,33,2116-12-24,first_admit_age,82.39,0 days 00:00:00.000000000,years
5,252348.0,61,2118-07-13,first_admit_age,54.65,0 days 00:00:00.000000000,years
6,216609.0,62,2113-02-18,first_admit_age,68.77,0 days 00:00:00.000000000,years
7,272667.0,68,2174-01-12,first_admit_age,41.79,0 days 00:00:00.000000000,years
8,284305.0,73,2170-12-23,first_admit_age,57.59,0 days 00:00:00.000000000,years
9,256068.0,80,2148-11-12,first_admit_age,43.91,0 days 00:00:00.000000000,years


time: 35 ms


In [115]:
#date= '22102018'
save_df(dfs['pt_info_df'], 'pt_info')

del(dfs['pt_info_df'] )

time: 284 ms
